# 処理パタン

結局のところ、処理パタンは決まっていて、それを呼び出すだけ。

詳細の理解は、別途、学習した方が良い。コードと理論の対応関係を大まかに理解させるために、
６００＋４００ページもの分厚い本が存在するのだと理解した。

## GPU設定

In [23]:
import torch

In [1]:
def get_device():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    print(f'device count: {torch.cuda.device_count()}')
    print(f'device name: {torch.cuda.get_device_name()}')
    print(f'device capability: {torch.cuda.get_device_capability()}')

    return device

## データ前処理

In [19]:
import torchvision.transforms as transforms

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### １階テンソル化

In [21]:
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Lambda(lambda x: x.view(-1)),
])

### 正規化のみ

In [22]:
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

## データローダ

### 訓練データと予測データの分割

キーワード引数stratifyは、２値分類で、訓練データと予測データの中にある、正例と負例の比率が同じになることを保証する。

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.7, test_size=0.3, random_state=123, stratify=True)

### ミニバッチ処理

ミニバッチ処理の最小単位のバッチのサイズをbatch_sizeで指定している。一般的に、この値が小さい程、並列性は低くなるが、精度は上がることが知られている。

In [ ]:
from torch.utils.data import DataLoader

batch_size = 500

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

## モデル定義

In [3]:
from torch import nn

In [4]:
class Net(nn.Module):
    '''
    隠れ層が１層のNN
    '''
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.l1 = nn.Linear(n_input, n_hidden)
        self.relu = nn.ReLU(inplace=True)
        self.l2 = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_input, n_hidden, n_output): # n_input == 32 * 14 * 14 = 6272. See p.344--p.347.
        super().__init__()
        # CIFAR-10データセット用の次元
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((2, 2))
        # １次元化
        self.flatten = nn.Flatten()
        # 上記Netと同様
        self.l1 = nn.Linear(n_input, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_output)

        self.features = nn.Sequential(
            self.conv1,
            self.relu,
            self.conv2,
            self.relu,
            self.maxpool,
        )
        self.classifier = nn.Sequential(
            self.l1,
            self.relu,
            self.l2,
        )

    def forward(self, x):
        x1 = self.features(x)
        x2 = self.flatten(x1)
        x3 = self.classifier(x2)

        return x3


## モデル表示

In [15]:
def print_model_parameters(net):
    for parameter in net.named_parameters():
        print(parameter)

## 損失計算（動作確認用）

In [14]:
from torchviz import make_dot

In [17]:
def show_computational_graph_for_loss(loader, device, net, criterion):
    # データローダから最初の１セット取得（計算グラフ作成のため）
    for images, labels in loader:
        break
    # デバイス（GPU/CPU割当）
    inputs = images.to(device)
    labels = labels.to(device)
    # 予測計算
    outputs = net(inputs)
    # 損失計算
    loss = criterion(outputs, labels)
    # 計算グラフの表示
    make_dot(loss, params=dict(net.named_parameters()))
    
    return loss

## 学習・追加学習用関数

In [8]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
history = np.zeros((0, 5))

＊ `net.train()`は「訓練中」、`net.eval()`は「予測中」の別をPyTorchに知らせる

In [9]:
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):
    base_epochs = len(history)

    for epoch in range(base_epochs, num_epochs + base_epochs):
        # 1エポックあたりの正解数（精度計算用）
        n_train_acc, n_val_acc = 0, 0
        # 1エポックあたりの累積損失（平均化前）
        train_loss, val_loss = 0, 0
        # 1エポックあたりのデータ数
        n_train, n_test = 0, 0

        ##### 訓練 #####
        net.train() # ドロップアウト関数nn.DropoutやBN関数nn.BatchNorm2dに訓練中であることを伝える関数

        for inputs, labels in tqdm(train_loader):
            train_batch_size = len(labels)
            n_train += train_batch_size

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            predicted = torch.max(outputs, 1)[1]
            train_acc += (predicted == labels).sum().item()
            train_loss += loss.item() * train_batch_size

        train_acc /= n_train
        train_loss /= n_train

        ##### 予測 #####
        net.eval() # ドロップアウト関数nn.DropoutやBN関数nn.BatchNorm2dに予測中であることを伝える関数

        for inputs_test, labels_test in test_loader:
            test_batch_size = len(labels_test)
            n_test += test_batch_size

            inputs_test = inputs_test.to(device)
            labels_test = labels_test.to(device)

            outputs_test = net(inputs_test)
            loss_test = criterion(outputs_test, labels_test)

            predict_test = torch.max(outputs_test, 1)[1]
            val_acc += (predict_test == labels_test).sum().item()
            val_loss += loss_test.item() * test_batch_size

        val_acc /= n_test
        val_loss /= n_test

        item = np.array([epoch, train_loss, train_acc, val_loss, val_acc])
        history = np.vstack((history, item))
        if epoch % (num_epochs / 100) == 0:
            print (f'Epoch [{epoch}/{num_epochs}], loss: {train_loss:.5f} acc: {train_acc:.5f} val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}')

    return history


## 学習ログ

### 学習前後でのlossとaccuracyを表示

In [12]:
def print_history_head_and_tail(history):
    print(f'Initial state: loss = {history[0, 3]:.5f} / accuracy = {history[0, 4]:.5f}')
    print(f'Final state: loss = {history[-1, 3]:.5f} / accuracy = {history[-1, 4]:.5f}')

### lossをepochの関数としてプロット

In [13]:
import matplotlib.pyplot as plt

In [14]:
def plot_loss_history(history):
    plt.plot(history[:, 0], history[:, 1], label='train loss')
    plt.plot(history[:, 0], history[:, 3], label='val loss')
    plt.legend()
    plt.title('Learning Curve wrt Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

### accuracyをepochの関数としてプロット

In [15]:
def plot_accuracy_history(history):
    plt.plot(history[:, 0], history[:, 2], label='train acc')
    plt.plot(history[:, 0], history[:, 4], label='val acc')
    plt.legend()
    plt.title('Learning Curve wrt Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()

## 予測結果の可視化

いちばん重要なところでありながら、本では幾つかが省略されている。著者のGitHubを見るも、全てを公開はしている訳ではない。

### 回帰の場合

一般的に、特徴量が２つ以上になるので、普通にはプロットできない。

ここでは、特徴量が２つの場合のみの予測結果の可視化関数を定義する。

In [1]:
import numpy as np

In [2]:
def show_regression(data, net):
    x = data[:, 0]
    y = data[:, 1]

    xse = np.array((x.min(), x.max())).reshape(-1, 1)
    Xse = torch.tensor(xse).float()

    with torch.no_grad():
        # 端っこの点を取って、それを結べば良いだけ（直線の場合はこれで良い）
        Yse = net(Xse)
    
    plt.scatter(x, y, s=10, c='b')
    plt.plot(Xse.data, Yse.data, c='k')
    plt.show()


### 分類の場合

- [sklearnの場合](https://www.dskomei.com/entry/2018/03/04/125249)
    - [著者による解説](https://qiita.com/makaishi2/items/2d0dd015ac1cbd86dd3f)

torchの場合も基本は同じ。`numpy.ravel`と`np.meshgrid`を使うとできる。[こちら](https://zenn.dev/curry/articles/b8ca6a3b668f3a)をまとめる。

### 画像データ分類の場合

In [ ]:
def show_images_labels(loader, classes, net, device):
    plt.figure(figsize=(10, 3))
    for images, labels in loader:
        num_show = min(20, len(labels))
        predicted = None
        if net is not None:
            net = net.to(device)
            predicted = net(images.to(device))
        for ii in range(num_show):
            ax = plt.subplot(2, 10, ii + 1)

            image = images[ii]
            label_name = classes[labels[ii]]

            plt.imshow(image)

            title_name = label_name
            color = 'k'

            if net is not None:
                predicted_name = classes[predicted[ii]]
                if label_name != predicted_name:
                    color = 'r'
                title_name = f'[{label_name}] {predicted_name}'

            ax.set_title(title_name, fontsize=20, c=color)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
        break
    plt.tight_layout()
    plt.show()


## 乱数を固定する（結果の再現性を担保するため）

In [7]:
def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # GPUで結果がdeterministicになるように
    torch.use_deterministic_algorithms = True

## モデルをシリアライズする

実用上、非常に重要なことなのに、この点についても振れられていないので、自分で調査した。

様々なパッケージがあり、どうやら確立された方法がないらしい。

- [cloudpickle](https://github.com/cloudpipe/cloudpickle): sagemakerも内部的に使っている（依存関係がある）
- [joblib](https://joblib.readthedocs.io/en/stable/): pickleより遅いが、ファイルサイズは小さくなる利点がある
- [h5py](https://docs.h5py.org/en/stable/): 科学技術計算での標準的フォーマットHDF5
- [dill](https://pypi.org/project/dill/): joblibと同様、pickleよりファイルサイズが小さくなるらしい

Python標準の`pickle`では、`torch.tensor`がもつ、内部状態を保存する際、
問題が発生するらしい。最終的に、独自関数が用意されていることが分かった。

- [torch.save/load](https://wandb.ai/wandb_fc/japanese/reports/PyTorch---VmlldzoxNTAyODQy)

### 学習済みモデル

#### 保存

state_dictを使うことが推奨されている。

In [ ]:
torch.save(net.state_dict(), 'save/to/path/model.pth')

#### 読込

state_dictを使う場合、以下のように、モデルの定義が必要になる。

In [ ]:
model = Net(n_input, n_hidden, n_output)
model.load_state_dict(torch.load('load/from/path/model.pth'))

### 学習途中で中断（チェックポイント）

#### 保存

In [ ]:
torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, 'save/to/path/model.pth')

#### 読込

In [ ]:
net = Net(n_input, n_hidden, n_output)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) ## just example

checkpoint = torch.load('load/from/path/model.pth')
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

## 使い方の例

### 変数の初期化

In [ ]:
n_input, n_hidden, n_output = 784, 100, 10

device = get_device()
net = Net(n_input, n_hidden, n_output).to(device)

lr = 0.01
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

criterion = nn.CrossEntropyLoss()

num_epochs = 100
history = np.zeros((0,5))

### モデル構築

In [ ]:
# 乱数初期化
torch_seed()

# 学習
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

### モデル確認

In [ ]:
print_model_parameters(net)

In [ ]:
show_computational_graph_for_loss(train_loader, device, net, criterion)

### 収束性の確認

In [ ]:
print_history_head_and_tail(history)

In [ ]:
plot_loss_history(history)

In [ ]:
plot_accuracy_history(history)

### 予測結果の可視化